In [1]:
import glob
import torch

from model import load_untrained_bert_base
from dataset_loading import load_tokenized_dataset, load_glue_sentence_classification
from finetune import run_epoch
from pathlib import Path

/local/scratch-3/fwe21/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_tokenized_dataset(
    Path("../../GLUE-baselines/glue_data/SST-2/"),
    load_glue_sentence_classification
)

In [3]:
def run_eval(model):
    loss, accuracy = run_epoch(model.module, torch.utils.data.DataLoader(
            dataset["dev"],
            batch_size=16,
        ), device="cuda")
    return loss, accuracy

In [4]:
from transformers import AutoModelForSequenceClassification
from model import get_bert_config

def print_checkpoints(path):
    checkpoints = sorted(glob.glob(str(path) + "checkpoint*"))
    for path in checkpoints:
        checkpoint = torch.load(path, map_location=torch.device("cpu"))
        print(f"Epochs: {checkpoint['epochs']}:")
        print("Train loss:",checkpoint["train_loss"], "\t\tTrain accuracy:",checkpoint["train_accuracy"])
        print("Test loss:",checkpoint["test_loss"], "\t\tTest accuracy:",checkpoint["test_accuracy"])

        print(checkpoint["optimizer_state_dict"]["param_groups"][0]["lr"])

        #if checkpoint['epochs']==3:
        
        #torch.save(model,"../../models/finetuned_bert_base_ss2_92.4%.pt")
        

In [14]:
from model import BertForSequenceClassificationWithLoss
from transformers import BertForSequenceClassification
checkpoint = torch.load("../../models/bert_base_SST-2_93.58%/checkpoint_epoch001", map_location=torch.device("cuda"))
config = get_bert_config("base")
model = AutoModelForSequenceClassification.from_config(config)

In [16]:

#print(checkpoint["model_state_dict"].keys())
model = torch.nn.DataParallel(BertForSequenceClassificationWithLoss(model))
model.load_state_dict(checkpoint["model_state_dict"])
model.to("cuda")

DataParallel(
  (module): BertForSequenceClassificationWithLoss(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): B

In [17]:
print(run_eval(model))

Loss: 0.2064, Acc: 93.58% device:cuda: 100%|██████████| 55/55 [00:09<00:00,  5.83it/s]

(tensor(0.2064, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.9358, device='cuda:0'))
